<a href="https://colab.research.google.com/github/AV-Gorshkov/Python_netology/blob/main/PY_%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D1%8F_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Задание***

 Использовать датасет MovieLens:
 - Построить рекомендации (регрессия, предсказываем оценку) на фичах:
    TF-IDF на тегах и жанрах;
    средние оценки (+ median, variance и т. д.) пользователя и фильма.
 - Оценить RMSE на тестовой выборке.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm_notebook

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#  загрузка данных
param = r'/content/drive/My Drive/Нетология/ML/Рекомендательные системы/'

links = pd.read_csv(f'{param}links.csv')
movies = pd.read_csv(f'{param}movies.csv')
ratings = pd.read_csv(f'{param}ratings.csv')
tags = pd.read_csv(f'{param}tags.csv')

In [ ]:
links.head()
# movies.head()
# ratings.head()
# tags.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
#  таблица с тегами
tags = tags[['movieId', 'tag']]
tags.head()

,movieId,tag
0,60756,funny
1,60756,Highly quotable
2,60756,will ferrell
3,89774,Boxing story
4,89774,MMA


In [ ]:
# соединим таблицу с рейтингами и названиями фильмов

joined_ratings = ratings.join(movies.set_index('movieId'), on='movieId')

joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [ ]:
#  преобразуем значение даты
joined_ratings['dt'] = joined_ratings['timestamp'].apply(lambda t: datetime.fromtimestamp(t))

joined_ratings['year'] = joined_ratings['dt'].dt.year
joined_ratings['month'] = joined_ratings['dt'].dt.month

joined_ratings.drop(['timestamp', 'dt'],axis= 1 , inplace= True )

joined_ratings.head(5)

,userId,movieId,rating,title,genres,year,month
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,2000,7
2,1,6,4.0,Heat (1995),Action|Crime|Thriller,2000,7
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,2000,7
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,2000,7


In [ ]:
# добави таблицу с тегами
joined_ratings = joined_ratings.join(tags.set_index('movieId'), on='movieId', lsuffix='_left', rsuffix='_right')

joined_ratings.head(5)

,userId,movieId,rating,title,genres,year,month,tag
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,pixar
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,pixar
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,fun
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,2000,7,moldy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,2000,7,old


In [ ]:
# средний рейтинг фильма

films_avg_rating = joined_ratings.groupby('title')[['rating']].mean().sort_values('rating', ascending=False)

films_avg_rating = films_avg_rating.rename( columns={'rating':'avg_rating'})
# медианнная оценка фильма

films_median_rating = joined_ratings.groupby('title')[['rating']].median().sort_values('rating', ascending=False)
films_median_rating  = films_median_rating .rename( columns={'rating':'median_rating'})
# films_median_rating.head(5)

films_avg_rating.head(5)

,avg_rating
title,
Gena the Crocodile (1969),5.0
True Stories (1986),5.0
Cosmic Scrat-tastrophe (2015),5.0
Love and Pigeons (1985),5.0
Red Sorghum (Hong gao liang) (1987),5.0


In [ ]:
# создадим словарь фильмов с кол-вом рейтингов
title_num_ratings = {}

for title, group in tqdm_notebook(joined_ratings.groupby('title')):

    title_num_ratings[title] = group.userId.unique().shape[0]

( [ (title_num_ratings[x], x ) for x in title_num_ratings])[:5]

  0%|          | 0/9719 [00:00<?, ?it/s]

[(1, "'71 (2014)"),
 (1, "'Hellboy': The Seeds of Creation (2004)"),
 (2, "'Round Midnight (1986)"),
 (1, "'Salem's Lot (2004)"),
 (2, "'Til There Was You (1997)")]

In [ ]:
# статистики по количеству рейтингов фильма

min_num_ratings = np.min([title_num_ratings[f] for f in title_num_ratings.keys()])
max_num_ratings = np.max([title_num_ratings[f] for f in title_num_ratings.keys()])
mean_num_ratings = np.mean([title_num_ratings[f] for f in title_num_ratings.keys()])
median_num_ratings = np.median([title_num_ratings[f] for f in title_num_ratings.keys()])

print(min_num_ratings)
print(max_num_ratings)
print(mean_num_ratings)
print(median_num_ratings)

# средний рейтинг на каждый фильм
title_mean_rating = {}

for title, group in tqdm_notebook(joined_ratings.groupby('title')):
    title_mean_rating[title] = group.rating.mean()

1
329
10.374729910484618
3.0


  0%|          | 0/9719 [00:00<?, ?it/s]

In [ ]:
# средневзвешенная оценка фильма: средняя оценка фильма, умноженная на нормированное количество рейтингов

film_with_our_mark = []

for f in title_num_ratings.keys():
    film_with_our_mark.append(
        (f, title_mean_rating[f] * (title_num_ratings[f] - mean_num_ratings) / (max_num_ratings - min_num_ratings))
    )

In [ ]:
# выводим топ 10 фильмов
list(sorted(film_with_our_mark, key=lambda x: x[1], reverse=True))[:10]

rating_films = pd.DataFrame(film_with_our_mark, columns=['title', 'cp_rating'])

In [ ]:
# добавим к исходной таблице новые признаки:

# среднее значение рейтинга фильма
joined_ratings = joined_ratings.merge(films_avg_rating, left_on = 'title', right_on='title', how = 'left' )

# медианное значение рейтинга фильма
joined_ratings = joined_ratings.merge(films_median_rating, left_on = 'title', right_on='title', how = 'left' )

# средневзыешенное значение рейтинга фильма
joined_ratings =joined_ratings.merge(rating_films, left_on = 'title', right_on='title', how = 'left' )

joined_ratings.head(5)

,userId,movieId,rating,title,genres,year,month,tag,avg_rating,median_rating,cp_rating
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,pixar,3.920930,4.0,2.446102
1,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,pixar,3.920930,4.0,2.446102
2,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,fun,3.920930,4.0,2.446102
3,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,2000,7,moldy,3.259615,3.0,0.413666
4,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,2000,7,old,3.259615,3.0,0.413666


In [ ]:
# средний рейтинг пользователя

user_avg_rating = joined_ratings.groupby('userId')[['rating']].mean().sort_values('rating', ascending=False)

user_avg_rating = user_avg_rating.rename( columns={'rating':'avg_rating'})
# медианнная оценка фильма

user_median_rating = joined_ratings.groupby('userId')[['rating']].median().sort_values('rating', ascending=False)
user_median_rating  = user_median_rating .rename( columns={'rating':'median_rating'})
# films_median_rating.head(5)

user_avg_rating.head(5)

,avg_rating
userId,
53,5.000000
251,4.969027
515,4.957711
348,4.907658
25,4.905028


In [ ]:
# создадим словарь user с кол-вом рейтингов пользователя
user_num_ratings = {}

for user, group in tqdm_notebook(joined_ratings.groupby('userId')):
    user_num_ratings[user] = group.movieId.count()

( [ (user_num_ratings[x], x ) for x in user_num_ratings])[:5]

  0%|          | 0/610 [00:00<?, ?it/s]

[(711, 1), (152, 2), (51, 3), (674, 4), (283, 5)]

In [ ]:
# статистики по количеству рейтингов пользователя

min_user_ratings = np.min([user_num_ratings[f] for f in user_num_ratings.keys()])
max_user_ratings = np.max([user_num_ratings[f] for f in user_num_ratings.keys()])
mean_user_ratings = np.mean([user_num_ratings[f] for f in user_num_ratings.keys()])
median_user_ratings = np.median([user_num_ratings[f] for f in user_num_ratings.keys()])

print(min_user_ratings)
print(max_user_ratings)
print(mean_user_ratings)
print(median_user_ratings)

# средний рейтинг на каждый фильм
user_mean_rating = {}

for user, group in tqdm_notebook(joined_ratings.groupby('userId')):
    user_mean_rating[user] = group.rating.mean()

21
4214
468.4622950819672
319.5


  0%|          | 0/610 [00:00<?, ?it/s]

In [ ]:
# средневзвешенная оценка пользователя

user_with_our_mark = []

for f in user_num_ratings.keys():
    user_with_our_mark.append(
        (f, user_mean_rating[f] * (user_num_ratings[f] - mean_user_ratings) / (max_user_ratings - min_user_ratings))
    )

rating_user = pd.DataFrame(user_with_our_mark, columns=['userId', 'cp_rating'])

# выводим топ 10
list(sorted(user_with_our_mark, key=lambda x: x[1], reverse=True))[:10]

[(414, 3.3228916622423808),
 (474, 2.4781665689130956),
 (599, 2.464238183106127),
 (610, 2.0942639949877875),
 (448, 1.9088095688346647),
 (380, 1.8277323974479807),
 (274, 1.8113969021805736),
 (249, 1.7958840292519331),
 (68, 1.5824487342981748),
 (606, 1.5063700081571767)]

In [ ]:
# добавим к исходной таблице новые признаки (рейтинг пользователя):

# среднее значение рейтинга
joined_ratings = joined_ratings.merge(user_avg_rating, left_on = 'userId', right_on='userId', how = 'left' )

# медианное значение рейтинга
joined_ratings = joined_ratings.merge(user_median_rating, left_on = 'userId', right_on='userId', how = 'left' )

# средневзыешенное значение рейтинга
joined_ratings =joined_ratings.merge(rating_user, left_on = 'userId', right_on='userId', how = 'left' )

joined_ratings.head(5)

,userId,movieId,rating,title,genres,year,month,tag,avg_rating_x,median_rating_x,cp_rating_x,avg_rating_y,median_rating_y,cp_rating_y
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,pixar,3.920930,4.0,2.446102,4.088608,4.0,0.236499
1,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,pixar,3.920930,4.0,2.446102,4.088608,4.0,0.236499
2,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2000,7,fun,3.920930,4.0,2.446102,4.088608,4.0,0.236499
3,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,2000,7,moldy,3.259615,3.0,0.413666,4.088608,4.0,0.236499
4,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance,2000,7,old,3.259615,3.0,0.413666,4.088608,4.0,0.236499


In [ ]:
#  преобразуем жанр фильма в векторы

def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))


film_genres = [change_string(g) for g in joined_ratings.genres.values]

tfidf = TfidfVectorizer()
film_tfidf = tfidf.fit_transform(film_genres)
# film_tfidf

df_film_tfidf = pd.DataFrame(film_tfidf.toarray(), columns=tfidf.get_feature_names_out())
df_film_tfidf.head()

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.0,0.348743,0.548508,0.527195,0.255409,0.0,0.0,0.0,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.348743,0.548508,0.527195,0.255409,0.0,0.0,0.0,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.348743,0.548508,0.527195,0.255409,0.0,0.0,0.0,0.484096,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.512716,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.858558,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,0.512716,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.858558,0.0,0.0,0.0,0.0


In [ ]:
#  преобразуем теги фильма в векторы

def change_string(s):
    return str(s).replace(' ', '').replace('-', '').lower()

tag_films = []
film = []

for movie, group in tqdm_notebook( joined_ratings.groupby('title')):

    tag_films.append(' '.join([change_string(s) for s in group.tag.values]))
    film.append(movie)

tfidf_tag = TfidfVectorizer()
tag_tfidf = tfidf.fit_transform(tag_films)
# # film_tfidf

df_tag_tfidf = pd.DataFrame(tag_tfidf.toarray(), columns=tfidf.get_feature_names_out())
df_tag_tfidf.head()

  0%|          | 0/9719 [00:00<?, ?it/s]

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# добавим новые признаки в датасет
df = joined_ratings.loc[:, ~joined_ratings.columns.isin(['title', 'genres', 'tag'])]

df = pd.merge(df, df_film_tfidf, left_index=True, right_index=True)

df = pd.merge(df, df_tag_tfidf, left_index=True, right_index=True)

df.head(5)

,userId,movieId,rating,year,month,avg_rating_x,median_rating_x,cp_rating_x,avg_rating_y,median_rating_y,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,1,4.0,2000,7,3.920930,4.0,2.446102,4.088608,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,4.0,2000,7,3.920930,4.0,2.446102,4.088608,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,4.0,2000,7,3.920930,4.0,2.446102,4.088608,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,4.0,2000,7,3.259615,3.0,0.413666,4.088608,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,3,4.0,2000,7,3.259615,3.0,0.413666,4.088608,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Построим модель регрессии

data = df.loc[:, df.columns != 'rating']
data_target = np.array(df['rating']) # целевая выборка

# разделяем выборку на тренировочную и тестовую

x_train, x_test, y_train, y_test = train_test_split(data, data_target, test_size=0.25, random_state=13)

# построим модель предсказания, с помощью логистической регрессии

lr = LinearRegression()
lr.fit(x_train, y_train)
lr.predict(x_test)

# точность модели

print('MSE Train: ', mean_squared_error(y_train, lr.predict(x_train) ) )
print('MSE Test: ', mean_squared_error(y_test, lr.predict(x_test) ) )

print('RMSE Train: ', mean_squared_error(y_train, lr.predict(x_train), squared = False) )
print('RMSE Test: ', mean_squared_error(y_test, lr.predict(x_test), squared = False) )

MSE Train:  0.7032972607775194
MSE Test:  0.8573386184173979
RMSE Train:  0.8386282017542216
RMSE Test:  0.9259258169083514
